# Bank Customer Classification
## Given a dataset consisiting of Bank Customer information, we are asked to build a classifier which will tell us if a customer will exit the bank or not.

In [1]:
import theano
import tensorflow
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import os
print(os.listdir("../input"))

['Churn_Modelling.csv']


# Data Preprocessing 

### In this dataset, we have to consider which of the factors may play a role in someone exiting a bank. To do that we must look at all  the column and infer whether it will matter in classifying a new customer or not.  The information about a customer is entailed in columns 0 through 12 (`EstimatedSalary`), while the output (whether the customer exited or not) is stored in the 13th row (`Exited`). 

In [2]:
#importing the dataset
dataset = pd.read_csv('../input/Churn_Modelling.csv')
dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


For as much as we care, neither the `CustomerID`, nor the `Surname` should matter in classification. 
Therefore, we will use columns 3 (`CreditScore`) inclusive through the 12th column (`EstimatedSalary`).

In [3]:
dataset.columns

Index(['RowNumber', 'CustomerId', 'Surname', 'CreditScore', 'Geography',
       'Gender', 'Age', 'Tenure', 'Balance', 'NumOfProducts', 'HasCrCard',
       'IsActiveMember', 'EstimatedSalary', 'Exited'],
      dtype='object')

In [4]:
X = dataset.loc[:,'CreditScore':'EstimatedSalary'].values # Credit Score through Estimated Salary
y = dataset.loc[:, 'Exited'].values # Exited

In [5]:
# Encoding categorical (string based) data. Country: there are 3 options: France, Spain and Germany
# This will convert those strings into scalar values for analysis
print(X[:8,1], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_country_encoder = LabelEncoder()
X[:,1] = label_X_country_encoder.fit_transform(X[:,1]) #The column of country
print(X[:8,1])

['France' 'Spain' 'France' 'France' 'Spain' 'Spain' 'France' 'Germany'] ... will now become: 
[0 2 0 0 2 2 0 1]


| Country |  -> | Country|
|------|      |------|  |------|    
|   France |   -> |0|
|   Spain | -> |2|
|   France | ->  |0|
|   France | -> |0|

In [6]:
# We will do the same thing for gender. this will be binary in this dataset
print(X[:6,2], '... will now become: ')
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
label_X_gender_encoder = LabelEncoder()
X[:,2] = label_X_gender_encoder.fit_transform(X[:,2])
print(X[:6,2])

['Female' 'Female' 'Female' 'Female' 'Female' 'Male'] ... will now become: 
[0 0 0 0 0 1]


The Problem here is that we are treating the countries as one variable with ordinal values (0 < 1 <  2). Therefore, one way to get rid of that problem is to split the countries into respective dimensions. that is,

| Country |  -> | Country|-> |France|Germany|Spain|
|------|      |------|  |------|    |------|    |------|
|   France |   -> |0| -> |1|0|0|
|   Spain | -> |1| -> |0|0|1|
|   Germany | ->  |1| -> |0|1|0|

In [7]:
# Converting the string features into their own dimensions. Gender doesn't matter here because its binary
countryhotencoder = OneHotEncoder(categorical_features = [1]) # 1 is the country column
X = countryhotencoder.fit_transform(X).toarray()

Gender doesn't need to go through Encoding because it is binary

In [8]:
X[:,0:3]

array([[ 1.,  0.,  0.],
       [ 0.,  0.,  1.],
       [ 1.,  0.,  0.],
       ..., 
       [ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 1.,  0.,  0.]])

You can now see that the first three columns represent the three countries that constituted the "country" category. We can now observe that  we essentially only need two columns: a 0 on two countries means that the country has to be the one variable which wasn't included. This will save us from the problem of using too many dimensions

|France|Germany|Spain|-> |Germany|Spain|
 |------|    |------|    |------|     |------|     |------|
 |1|0|0|-> |0|0|
|0|0|1|-> |0|1|
|0|1|0|-> |1|0|

In [9]:
X = X[:,1:] # Got rid of France as a dimension. It is still there through our inferences

In [10]:
# Splitting the dataset into the Training and Testing set.
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size = 0.2, random_state = 0)

Feature scaling is a method used to standardize the range of independent variables or features of data. It is basically scaling all the dimensions to be even so that one independent variable does not dominate another. For example, bank account balance ranges from millions to 0, whereas gender is either 0 or 1. If one of the features has a broad range of values, the distance will be governed by this particular feature. Therefore, the range of all features should be normalized so that each feature contributes approximately proportionately to the final distance.

In [11]:
# Feature Scaling
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
X_train

array([[-0.5698444 ,  1.74309049,  0.16958176, ...,  0.64259497,
        -1.03227043,  1.10643166],
       [ 1.75486502, -0.57369368, -2.30455945, ...,  0.64259497,
         0.9687384 , -0.74866447],
       [-0.5698444 , -0.57369368, -1.19119591, ...,  0.64259497,
        -1.03227043,  1.48533467],
       ..., 
       [-0.5698444 , -0.57369368,  0.9015152 , ...,  0.64259497,
        -1.03227043,  1.41231994],
       [-0.5698444 ,  1.74309049, -0.62420521, ...,  0.64259497,
         0.9687384 ,  0.84432121],
       [ 1.75486502, -0.57369368, -0.28401079, ...,  0.64259497,
        -1.03227043,  0.32472465]])

## END OF PREPROCESSING

## Making the ANN

In [25]:
import keras # Test out Theano when time permits as well

Using TensorFlow backend.


In [18]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout

In [31]:
# Initializing the ANN
classifier = Sequential()

A hurestic tip is that the amount of nodes (dimensions) in your hidden layer should be the average of your input and output layers, which means that since we have 11 dimensions (representing **Independent variables** Note: Countries still compose only  **one** dimension) and we are looking for a binary output, we calculate this to be $(11+1)\div 2 = 6 $.

#### The breakdown of the inputs for the first layer is as follows:

*activiation*: **relu** becasue we are in an input layer. uses the ReLu activation function for $\phi$

*input_dim*: **11** because we span 11 dimensions in our input layer. This is needed for the first added layer. The subsequent layers's input dimensions can be inferred using the previously added layer's output dimension. The next hidden layer will know what to expect.

*units*: **6** nodes (number of nodes in hidden layer). Can think of this as number of nodes are in the next layer.

*kernel_initializer*: **uniform**  the distribution with which we randomly initialize weights for the nodes in this layer. 

Dropout:

*rate*: **0.1** the fraction of neuron we want disabled randomly


In [32]:
# This adds the input layer (by specifying input dimension) AND the first hidden layer (units) with Dropout
classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))
classifier.add(Dropout(rate=0.1))

### We are going to add another layer to this model because we want to implement Deep Learning, which is an artificial Neural network with many layers.
We will make our second hidden layer also have 6 nodes, just playing with the same arithmetic we used to determine the dimensions of the first hidden layer (average of your input and output layers) $(11+1)\div 2 = 6 $.

In [33]:
# Adding the second hidden layer
# Notice that we do not need to specify input dim. 
classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 

### Adding the output layer
#### The breakdown of the inputs for the output layer is as follows:

*activiation*: **sigmoid** becasue we are in an output layer. uses the Sigmoid activation function for $\phi$. This is used instead of the ReLu function becasue it generates probabilities for the outcome. We want the probability that each customer leaves the bank.  

*units*: **1** nodes (number of nodes in output layer). This will be the probability that a customer leaves.

*kernel_initializer*: **uniform**  the distribution with which we randomly initialize weights for the nodes in this layer. 

In [34]:
# Adding the output layer
# Notice that we do not need to specify input dim. 
# we have an output of 1 node, which is the the desired dimensions of our output (stay with the bank or not)
# We use the sigmoid because we want probability outcomes
classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 


### If we want more than two categories, then we will need to change 

 1) the *units* parameter to match the desired category count (In this example category: "will customer leave or not")
 
 2) the *activation* field to **softmax**.  Basically a sigmoid function but applied to a dependent variable that has more than 2 categories.

## Compiling the Neural Network
Basically applying Stochastic Gradient descent on the whole Neural Network. We are Tuning the individual weights on each neuron.

#### The breakdown of the inputs for compiling is as follows:

*optimizer*: ** adam** The algorithm we want to use to find the optimal set of weights in the neural networks.  Adam is a very efficeint variation of Stochastic Gradient Descent.

*loss*: **binary_crossentropy** This is the loss function used within adam. This should be the logarthmic loss. If our dependent (output variable) is *Binary*, it is **binary_crossentropy**. If *Categorical*, then it is called **categorical_crossentropy**

*metrics*: **[accuracy]** The accuracy metrics which will be evaluated(minimized) by the model. Used as accuracy criteria to imporve model performance. 

*kernel_initializer*: **uniform**  the distribution with which we randomly initialize weights for the nodes in this layer. 

In [35]:
classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])

## Fitting the Neural Network
This is where we will be fitting the ANN to our training set.

#### The breakdown of the inputs for compiling is as follows:

** X_train** The independent variable portion of the data which needs to be fitted with the model.

** Y_train** The output portion of the data which the model needs to produce after fitting.

*batch_size*:  How often we want to back-propogate the error values so that individual node weights can be adjusted. 

*nb_epochs*: The number of times we want to run the entire test data over again to tune the weights. This is like the fuel of the algorithm. 

In [36]:
classifier.fit(X_train, y_train, batch_size=10, epochs=100)

Epoch 1/100
8000/8000 [==============================] - 2s 242us/step - loss: 0.4940 - acc: 0.7959
Epoch 2/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.4331 - acc: 0.7960
Epoch 3/100
8000/8000 [==============================] - 2s 235us/step - loss: 0.4261 - acc: 0.7960
Epoch 4/100
8000/8000 [==============================] - 2s 232us/step - loss: 0.4258 - acc: 0.7960
Epoch 5/100
8000/8000 [==============================] - 2s 227us/step - loss: 0.4237 - acc: 0.8185
Epoch 6/100
8000/8000 [==============================] - 2s 237us/step - loss: 0.4209 - acc: 0.8266
Epoch 7/100
8000/8000 [==============================] - 2s 234us/step - loss: 0.4172 - acc: 0.8304
Epoch 8/100
8000/8000 [==============================] - 2s 246us/step - loss: 0.4160 - acc: 0.8311
Epoch 9/100
8000/8000 [==============================] - 2s 240us/step - loss: 0.4151 - acc: 0.8314
Epoch 10/100
8000/8000 [==============================] - 2s 241us/step - loss: 0.4145 - acc: 0.8295

Epoch 82/100
8000/8000 [==============================] - 3s 337us/step - loss: 0.4034 - acc: 0.8365
Epoch 83/100
8000/8000 [==============================] - 3s 361us/step - loss: 0.4013 - acc: 0.8387
Epoch 84/100
8000/8000 [==============================] - 3s 333us/step - loss: 0.4035 - acc: 0.8379
Epoch 85/100
8000/8000 [==============================] - 2s 304us/step - loss: 0.4049 - acc: 0.8380
Epoch 86/100
8000/8000 [==============================] - 2s 306us/step - loss: 0.4021 - acc: 0.8389
Epoch 87/100
8000/8000 [==============================] - 3s 324us/step - loss: 0.3997 - acc: 0.8420
Epoch 88/100
8000/8000 [==============================] - 3s 337us/step - loss: 0.4040 - acc: 0.8374
Epoch 89/100
8000/8000 [==============================] - 2s 301us/step - loss: 0.4009 - acc: 0.8386
Epoch 90/100
8000/8000 [==============================] - 2s 287us/step - loss: 0.4038 - acc: 0.8381
Epoch 91/100
8000/8000 [==============================] - 2s 283us/step - loss: 0.4015 - ac

The output network should converge to an accuracy near 83%
## Testing the ANN
### Predicting the Test set results

This shows the probability of a customer leaving given the testing data. Each row in X_test corresponds to a row in Y_test

In [37]:
y_pred = classifier.predict(X_test)
y_pred[:8]

array([[ 0.24225244],
       [ 0.28853273],
       [ 0.17794316],
       [ 0.06567096],
       [ 0.18370007],
       [ 0.81447601],
       [ 0.06782628],
       [ 0.078195  ]], dtype=float32)

To use the confusion Matrix, we need to convert the probabilities that a customer will leave the bank into the form true or false. So we will use the cutoff value 0.5 to indicate whether they are likely to exit or not.

In [38]:
y_pred = (y_pred > 0.5)
y_pred[:8]

array([[False],
       [False],
       [False],
       [False],
       [False],
       [ True],
       [False],
       [False]], dtype=bool)

### Making the Confusion Matrix

In [39]:
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)

[[1545   50]
 [ 266  139]]


### Significance of the confusion matrix value:

The output should be close to the table below:

||Predicted: No |Predicted: Yes|
 |------| |------|   |------|  
 |Actual: No|1545|50|
|Actual: Yes|266|139|


This means that we should have about $(1545 + 139) = 1684$ correct classifications out of our total testing data size of $2000$.
This means that our accuracy for this trial was $1686 \div 2000 = 84.2\%$, which matches the classifier's prediction

In [40]:
print (((cm[0][0]+cm[1][1])*100)/(cm[0][0]+cm[1][1]+cm[0][1]+cm[1][0]), '% of testing data was classified correctly')

84.2 % of testing data was classified correctly


## Testing on a random new user

using the ANN model to predict if the customer with the following informations will leave the bank: 

* Geography: France
* Credit Score: 600
* Gender: Male
* Age: 40 years old
* Tenure: 3 years
* Balance: \$60000
* Number of Products: 2
* Does this customer have a credit card ? Yes
* Is this customer an Active Member: Yes
* Estimated Salary: $50000
* So should we say goodbye to that customer ?

In [41]:
new_user_x = np.array([[0,0, # France
              600, # Credit score
              0, # Male
              40, # years old
              3, # tenure
              60_000.0, # balance
              2, # Number of products
              1, # Credit Card
              1, # Active
              50_000 # Salary
             ]])

In [42]:
new_user_x_norm = sc.transform(new_user_x)

In [43]:
print(classifier.predict(new_user_x_norm)[0][0]*100,'% probability that customer will leave') 

17.613402009 % probability that customer will leave


## Evaluating, Improving and Tuning the ANN
### Evaluating the ANN

In [13]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score

Using TensorFlow backend.


In [21]:
def build_classifier():
    classifier = Sequential()
    classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='uniform'))
    classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='uniform')) 
    classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='uniform')) 
    classifier.compile(optimizer='adam', loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

*build_fn*: ** build_classifier** The function which is used to build the classifier.

*batch_size*: **10** Same as what we used to generate our classifier

*nb_epoch*: **100** Same as what we used to generate our classifier

In [22]:
classifier = KerasClassifier(build_fn=build_classifier, batch_size=10, epochs=100)

cross_val_score will be responsible for returning a relevant accuracy measure. This will measure the accuracy across our folds (10)
*estimator*: ** classifier** The object to use to fit the data.

*X*: **10** The features or the Matrix. the X portion of our data. The data to fit

*Y*: **100** The target variable to try to predict in supervised learning

*cv*: **10** Number of folds are used. 

*n_jobs*: **-1** Use all CPUs to run the validation

In [23]:
accuracies = cross_val_score(estimator = classifier, X = X_train, y = y_train, cv = 10, n_jobs=-1)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 6s 803us/step - loss: 0.4778 - acc: 0.7971
Epoch 2/100
7200/7200 [==============================] - 6s 817us/step - loss: 0.4780 - acc: 0.7956
Epoch 2/100
7200/7200 [==============================] - 5s 735us/step - loss: 0.4283 - acc: 0.7967
Epoch 3/100
7200/7200 [==============================] - 5s 737us/step - loss: 0.4285 - acc: 0.7969
Epoch 3/100
7200/7200 [==============================] - 6s 769us/step - loss: 0.4294 - acc: 0.7956
Epoch 3/100
7200/7200 [==============================] - 6s 765us/step - loss: 0.4267 - acc: 0.7962
Epoch 3/100
7200/7200 [==============================] - 6s 778us/step - loss: 0.4289 - acc: 0.7944
Epoch 3/100
7200/7200 [==============================] - 5s 751us/step - loss: 0.4229 - acc: 0.7967
Epoch 4/100
7200/7200 [==============================] - 5s 763us/step - loss: 0.4200 - acc: 0.8021
Epoch 4/100
7200

/Users/tarunsunkaraneni/miniconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.122598). Check your callbacks.
  % delta_t_median)


7200/7200 [==============================] - 6s 840us/step - loss: 0.4119 - acc: 0.8314
Epoch 9/100
7200/7200 [==============================] - 6s 865us/step - loss: 0.4100 - acc: 0.8332
Epoch 9/100
7090/7200 [============================>.] - ETA: 0s - loss: 0.4138 - acc: 0.8313

/Users/tarunsunkaraneni/miniconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.120047). Check your callbacks.
  % delta_t_median)


7200/7200 [==============================] - 6s 868us/step - loss: 0.4152 - acc: 0.8300
Epoch 9/100
7200/7200 [==============================] - 6s 864us/step - loss: 0.4120 - acc: 0.8307
Epoch 9/100
7200/7200 [==============================] - 5s 633us/step - loss: 0.4102 - acc: 0.8304
Epoch 11/100
7200/7200 [==============================] - 5s 644us/step - loss: 0.4067 - acc: 0.8347
Epoch 11/100
Epoch 11/100
7200/7200 [==============================] - 5s 643us/step - loss: 0.4081 - acc: 0.8335
Epoch 11/100
7200/7200 [==============================] - 5s 639us/step - loss: 0.4109 - acc: 0.8319
Epoch 11/100
7200/7200 [==============================] - 5s 632us/step - loss: 0.4084 - acc: 0.8314
Epoch 13/100
7200/7200 [==============================] - 5s 685us/step - loss: 0.4072 - acc: 0.8347
Epoch 16/100
7200/7200 [==============================] - 5s 693us/step - loss: 0.4046 - acc: 0.8340
Epoch 16/100
7200/7200 [==============================] - 5s 700us/step - loss: 0.4074 - acc:

7200/7200 [==============================] - 6s 772us/step - loss: 0.4014 - acc: 0.8365
Epoch 46/100
7200/7200 [==============================] - 5s 745us/step - loss: 0.4010 - acc: 0.8357
Epoch 47/100
7200/7200 [==============================] - 5s 745us/step - loss: 0.4021 - acc: 0.8339
Epoch 47/100
7200/7200 [==============================] - 5s 734us/step - loss: 0.3981 - acc: 0.8353
Epoch 47/100
7200/7200 [==============================] - 5s 737us/step - loss: 0.3986 - acc: 0.8349
Epoch 48/100
7200/7200 [==============================] - 5s 744us/step - loss: 0.4010 - acc: 0.8353
Epoch 48/100
7200/7200 [==============================] - 5s 746us/step - loss: 0.4011 - acc: 0.8369
Epoch 49/100
7200/7200 [==============================] - 6s 765us/step - loss: 0.4020 - acc: 0.8339
Epoch 50/100
7200/7200 [==============================] - 5s 753us/step - loss: 0.3979 - acc: 0.8358
Epoch 50/100
7200/7200 [==============================] - 5s 759us/step - loss: 0.3991 - acc: 0.8362
Epo

Epoch 65/100
7200/7200 [==============================] - 6s 766us/step - loss: 0.4011 - acc: 0.8350
Epoch 67/100
7200/7200 [==============================] - 5s 762us/step - loss: 0.3972 - acc: 0.8375
Epoch 67/100
7200/7200 [==============================] - 5s 756us/step - loss: 0.4038 - acc: 0.8328
Epoch 68/100
7200/7200 [==============================] - 6s 772us/step - loss: 0.4013 - acc: 0.8350
Epoch 68/100
7200/7200 [==============================] - 6s 770us/step - loss: 0.4008 - acc: 0.8365
Epoch 71/100
7200/7200 [==============================] - 6s 777us/step - loss: 0.4002 - acc: 0.8376
Epoch 72/100
7200/7200 [==============================] - 6s 771us/step - loss: 0.4009 - acc: 0.8344
Epoch 72/100
7200/7200 [==============================] - 6s 783us/step - loss: 0.3974 - acc: 0.8365
Epoch 73/100
7200/7200 [==============================] - 6s 795us/step - loss: 0.3981 - acc: 0.8364
Epoch 73/100
7200/7200 [==============================] - 6s 800us/step - loss: 0.4034 - ac

7200/7200 [==============================] - 5s 651us/step - loss: 0.4032 - acc: 0.8333
Epoch 1/100
7200/7200 [==============================] - 5s 630us/step - loss: 0.4851 - acc: 0.7957
Epoch 2/100
7200/7200 [==============================] - 4s 566us/step - loss: 0.4860 - acc: 0.7958
Epoch 2/100
7200/7200 [==============================] - 3s 452us/step - loss: 0.4250 - acc: 0.7957
Epoch 3/100
7200/7200 [==============================] - 3s 442us/step - loss: 0.4292 - acc: 0.7961
Epoch 3/100
7200/7200 [==============================] - 3s 410us/step - loss: 0.4197 - acc: 0.7978
Epoch 4/100
7200/7200 [==============================] - 3s 411us/step - loss: 0.4242 - acc: 0.7964
Epoch 4/100
7200/7200 [==============================] - 3s 352us/step - loss: 0.4158 - acc: 0.8236
Epoch 5/100
7200/7200 [==============================] - 3s 350us/step - loss: 0.4205 - acc: 0.8210
Epoch 5/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.4138 - acc: 0.8274
Epoch 6/100


7200/7200 [==============================] - 2s 328us/step - loss: 0.4032 - acc: 0.8326
Epoch 77/100
7200/7200 [==============================] - 2s 324us/step - loss: 0.3936 - acc: 0.8374
Epoch 78/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.4033 - acc: 0.8358
Epoch 78/100
7200/7200 [==============================] - 2s 324us/step - loss: 0.3938 - acc: 0.8374
Epoch 79/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.4034 - acc: 0.8351
Epoch 79/100
7200/7200 [==============================] - 2s 325us/step - loss: 0.3938 - acc: 0.8376
Epoch 80/100
7200/7200 [==============================] - 2s 328us/step - loss: 0.4032 - acc: 0.8342
Epoch 80/100
7200/7200 [==============================] - 2s 326us/step - loss: 0.3935 - acc: 0.8383
Epoch 81/100
7200/7200 [==============================] - 2s 330us/step - loss: 0.4032 - acc: 0.8336
Epoch 81/100
7200/7200 [==============================] - 2s 329us/step - loss: 0.3935 - acc: 0.8378
Epo

In [24]:
accuracies

array([ 0.8325    ,  0.83499999,  0.83624999,  0.82375   ,  0.85499999,
        0.83375   ,  0.83625   ,  0.82624999,  0.8125    ,  0.84249999])

In [25]:
accuracies.mean()

0.83337499488145128

In [26]:
accuracies.var()

0.00011501560074650028

This has a decent accuracy and very low variance
### Tuning the ANN

In [14]:
from sklearn.model_selection import GridSearchCV

In [15]:
def build_classifier(optimizer):
    classifier = Sequential()
    classifier.add(Dense(activation = 'relu', input_dim = 11, units=6, kernel_initializer='glorot_uniform'))
    classifier.add(Dropout(rate=0.1))
    classifier.add(Dense(activation = 'relu', units=6, kernel_initializer='glorot_uniform')) 
    classifier.add(Dense(activation = 'sigmoid', units=1, kernel_initializer='glorot_uniform')) 
    classifier.compile(optimizer=optimizer, loss = 'binary_crossentropy', metrics=['accuracy'])
    return classifier

In [16]:
classifier = KerasClassifier(build_fn=build_classifier)
parameters = {'batch_size':[25],
              'epochs': [100],
              'optimizer': ['adam', 'rmsprop']}
grid_search = GridSearchCV(estimator=classifier, param_grid= parameters, scoring = 'accuracy', cv = 10, n_jobs=-1) # cv = folds

In [20]:
grid_search = grid_search.fit(X_train, y_train)

Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
Epoch 1/100
7200/7200 [==============================] - 2s 327us/step - loss: 0.5996 - acc: 0.7400
Epoch 2/100
7200/7200 [==============================] - 2s 333us/step - loss: 0.5938 - acc: 0.7458
Epoch 2/100
7200/7200 [==============================] - 2s 337us/step - loss: 0.5987 - acc: 0.7413
Epoch 2/100
7200/7200 [==============================] - 2s 345us/step - loss: 0.5982 - acc: 0.7418
Epoch 2/100
7200/7200 [==============================] - 2s 345us/step - loss: 0.5947 - acc: 0.7458
Epoch 2/100
7200/7200 [==============================] - 2s 286us/step - loss: 0.4903 - acc: 0.7967
Epoch 3/100
7200/7200 [==============================] - 2s 285us/step - loss: 0.4890 - acc: 0.7981
Epoch 3/100
Epoch 3/100
7200/7200 [==============================] - 2s 282us/step - loss: 0.4898 - acc: 0.7974
Epoch 3/100
7200/7200 [==============================] - 2s 292us/step - loss: 0.4738 - acc: 0.7960
Epoc

7200/7200 [==============================] - 2s 329us/step - loss: 0.4320 - acc: 0.8017
Epoch 9/100
7200/7200 [==============================] - 2s 316us/step - loss: 0.4270 - acc: 0.8040
Epoch 9/100
7200/7200 [==============================] - 2s 304us/step - loss: 0.4297 - acc: 0.8026
Epoch 10/100
7200/7200 [==============================] - 2s 296us/step - loss: 0.4279 - acc: 0.8068
Epoch 10/100
7200/7200 [==============================] - 2s 295us/step - loss: 0.4215 - acc: 0.8065
Epoch 10/100
7200/7200 [==============================] - 2s 320us/step - loss: 0.4223 - acc: 0.8054
Epoch 10/100
7200/7200 [==============================] - 2s 316us/step - loss: 0.4157 - acc: 0.8064
Epoch 11/100
7200/7200 [==============================] - 2s 334us/step - loss: 0.4229 - acc: 0.8074
Epoch 11/100
7200/7200 [==============================] - 2s 324us/step - loss: 0.4083 - acc: 0.8093
Epoch 11/100
7200/7200 [==============================] - 2s 338us/step - loss: 0.4148 - acc: 0.8079
Epoch

7200/7200 [==============================] - 3s 479us/step - loss: 0.3638 - acc: 0.8483
Epoch 24/100
7200/7200 [==============================] - 3s 452us/step - loss: 0.3716 - acc: 0.8425
Epoch 25/100
7200/7200 [==============================] - 3s 426us/step - loss: 0.3673 - acc: 0.8481
Epoch 25/100
7200/7200 [==============================] - 3s 432us/step - loss: 0.3675 - acc: 0.8454
Epoch 25/100
7200/7200 [==============================] - 3s 425us/step - loss: 0.3738 - acc: 0.8403
Epoch 25/100
7200/7200 [==============================] - 3s 421us/step - loss: 0.3738 - acc: 0.8412
Epoch 25/100
7200/7200 [==============================] - 3s 436us/step - loss: 0.3672 - acc: 0.8474
Epoch 25/100
7200/7200 [==============================] - 3s 412us/step - loss: 0.3707 - acc: 0.8465
Epoch 26/100
7200/7200 [==============================] - 3s 436us/step - loss: 0.3734 - acc: 0.8450
Epoch 26/100
7200/7200 [==============================] - 3s 438us/step - loss: 0.3692 - acc: 0.8487
Epo

7200/7200 [==============================] - 3s 437us/step - loss: 0.3651 - acc: 0.8496
Epoch 40/100
7200/7200 [==============================] - 3s 422us/step - loss: 0.3612 - acc: 0.8537
Epoch 41/100
7200/7200 [==============================] - 3s 431us/step - loss: 0.3641 - acc: 0.8464
Epoch 41/100
7200/7200 [==============================] - 3s 441us/step - loss: 0.3566 - acc: 0.8507
Epoch 41/100
7200/7200 [==============================] - 3s 423us/step - loss: 0.3581 - acc: 0.8529
Epoch 41/100
7200/7200 [==============================] - 3s 403us/step - loss: 0.3637 - acc: 0.8500
Epoch 41/100
7200/7200 [==============================] - 4s 487us/step - loss: 0.3609 - acc: 0.8504
Epoch 45/100
7200/7200 [==============================] - 4s 488us/step - loss: 0.3605 - acc: 0.8549
Epoch 46/100
7200/7200 [==============================] - 4s 503us/step - loss: 0.3626 - acc: 0.8512
Epoch 45/100
7200/7200 [==============================] - 3s 485us/step - loss: 0.3589 - acc: 0.8519
Epo

/Users/tarunsunkaraneni/miniconda3/lib/python3.6/site-packages/keras/callbacks.py:120: UserWarning: Method on_batch_end() is slow compared to the batch update (0.118070). Check your callbacks.
  % delta_t_median)


...

7200/7200 [==============================] - 1s 164us/step - loss: 0.3573 - acc: 0.8524
Epoch 42/100
7200/7200 [==============================] - 1s 175us/step - loss: 0.3603 - acc: 0.8504
Epoch 40/100
7200/7200 [==============================] - 1s 176us/step - loss: 0.3629 - acc: 0.8519
Epoch 41/100
7200/7200 [==============================] - 1s 176us/step - loss: 0.3696 - acc: 0.8486
Epoch 41/100
7200/7200 [==============================] - 1s 171us/step - loss: 0.3630 - acc: 0.8504
Epoch 43/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3610 - acc: 0.8511
Epoch 41/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.3666 - acc: 0.8469
Epoch 42/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.3552 - acc: 0.8540
Epoch 42/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3633 - acc: 0.8518
Epoch 43/100
7200/7200 [==============================] - 1s 180us/step - loss: 0.3684 - acc: 0.8467
Epo

7200/7200 [==============================] - 1s 186us/step - loss: 0.3567 - acc: 0.8517
Epoch 59/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.3596 - acc: 0.8526
Epoch 59/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.3573 - acc: 0.8519
Epoch 62/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3530 - acc: 0.8528
Epoch 59/100
7200/7200 [==============================] - 1s 188us/step - loss: 0.3619 - acc: 0.8501
Epoch 60/100
7200/7200 [==============================] - 1s 176us/step - loss: 0.3587 - acc: 0.8494
Epoch 63/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3624 - acc: 0.8504
Epoch 61/100
7200/7200 [==============================] - 1s 177us/step - loss: 0.3690 - acc: 0.8482
Epoch 61/100
7200/7200 [==============================] - 1s 166us/step - loss: 0.3558 - acc: 0.8532
Epoch 64/100
7200/7200 [==============================] - 1s 172us/step - loss: 0.3597 - acc: 0.8510
Epo

7200/7200 [==============================] - 1s 171us/step - loss: 0.3659 - acc: 0.8489
Epoch 77/100
7200/7200 [==============================] - 1s 183us/step - loss: 0.3591 - acc: 0.8532
Epoch 78/100
7200/7200 [==============================] - 1s 175us/step - loss: 0.3547 - acc: 0.8543
Epoch 81/100
7200/7200 [==============================] - 1s 185us/step - loss: 0.3582 - acc: 0.8531
Epoch 77/100
7200/7200 [==============================] - 1s 178us/step - loss: 0.3585 - acc: 0.8522
Epoch 79/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.3576 - acc: 0.8526
Epoch 78/100
7200/7200 [==============================] - 1s 174us/step - loss: 0.3624 - acc: 0.8524
Epoch 79/100
7200/7200 [==============================] - 1s 164us/step - loss: 0.3572 - acc: 0.8522
Epoch 83/100
7200/7200 [==============================] - 1s 173us/step - loss: 0.3545 - acc: 0.8546
Epoch 80/100
7200/7200 [==============================] - 1s 173us/step - loss: 0.3542 - acc: 0.8558
Epo

7200/7200 [==============================] - 1s 172us/step - loss: 0.3546 - acc: 0.8528
Epoch 99/100
7200/7200 [==============================] - 1s 186us/step - loss: 0.3559 - acc: 0.8544
Epoch 96/100
7200/7200 [==============================] - 1s 187us/step - loss: 0.3530 - acc: 0.8544
Epoch 95/100
7200/7200 [==============================] - 1s 184us/step - loss: 0.3584 - acc: 0.8483
Epoch 96/100
7200/7200 [==============================] - 1s 181us/step - loss: 0.3501 - acc: 0.8537
Epoch 100/100
7200/7200 [==============================] - 1s 189us/step - loss: 0.3591 - acc: 0.8514
Epoch 97/100
7200/7200 [==============================] - 1s 190us/step - loss: 0.3527 - acc: 0.8562
Epoch 96/100
7200/7200 [==============================] - 1s 189us/step - loss: 0.3630 - acc: 0.8490
Epoch 97/100
7200/7200 [==============================] - 1s 163us/step - loss: 0.3555 - acc: 0.8543
Epoch 98/100
7200/7200 [==============================] - 1s 158us/step - loss: 0.3574 - acc: 0.8532
Ep

8000/8000 [==============================] - 1s 83us/step - loss: 0.3586 - acc: 0.8517
Epoch 62/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3671 - acc: 0.8486
Epoch 63/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3612 - acc: 0.8512
Epoch 64/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3590 - acc: 0.8559
Epoch 65/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3602 - acc: 0.8515
Epoch 66/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3602 - acc: 0.8504
Epoch 67/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3619 - acc: 0.8512
Epoch 68/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3589 - acc: 0.8525
Epoch 69/100
8000/8000 [==============================] - 1s 83us/step - loss: 0.3554 - acc: 0.8551
Epoch 70/100
8000/8000 [==============================] - 1s 84us/step - loss: 0.3597 - acc: 0.8525
Epoch 71/100


In [21]:
best_parameters = grid_search.best_estimator_ 
best_accuracy = grid_search.best_score_ 

In [22]:
grid_search.best_params_

{'batch_size': 25, 'epochs': 100, 'optimizer': 'rmsprop'}

### This shows us that the 

In [23]:
grid_search.best_score_ 

0.85812500000000003